In [3]:
from lxml import etree
import requests 
import re
from bs4 import BeautifulSoup as BS
import pymysql

In [36]:
def get_html(url):
    headers={"user-agent":'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:70.0) Gecko/20100101 Firefox/70.0'}
    response=requests.get(url,headers=headers)   
    if response.status_code==200:
        return response.content
def get_sub_information(url):
    html=get_html(url).decode("gbk")
    soup = BS(html, "html.parser")
    #find information
    if len(soup.find_all('p', class_="msg ltype"))>=1:
        infor=soup.find_all('p', class_="msg ltype")[0]["title"]
    else: 
        return []
    infor_list=[]
    for ainfor in infor.split("|")[:5]:
        infor_list.append(ainfor.strip())
    company_name=soup.find('div', class_="tHeader tHjob").find("p",class_="cname").find("a").text
    #print(company_name)
    infor_list.append(company_name)
    return infor_list

def create_table():
    conn = pymysql.connect(host="localhost", user="root", password="xs448058", db="h4", charset="utf8")
    cursor = conn.cursor()
    create_sql = """
        create table if not exists `jobs512`(
        `id` int(11) not null primary key auto_increment,
        `position` varchar(255) not null,
        `company` varchar(255) not null,
        `location` varchar(255) not null,
        `experience` varchar(255) not null,        
        `education` varchar(255) not null,
        `numberofrecruitment` varchar(255) not null,
        `date` varchar(255) not null,
        `salary` varchar(255) not null)
        """
    cursor.execute(create_sql)
    conn.commit()
    cursor.close()
    conn.close()

def save_to_mysql(info_list):
    conn=pymysql.connect(host="localhost",user="root",password="xs448058",db="h4",charset="utf8")
    cursor=conn.cursor()
    insert_sql="insert into `jobs511` (position,company,location,experience,education,numberofrecruitment,date,salary) values(%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute('SET NAMES utf8')
    for line in info_list:
        try:
            cursor.execute(insert_sql,line)
        except Exception as e:
            print(e)
    conn.commit()
    cursor.close()
    conn.close()
    
def phase_html(html,num):
    info_list=[]
    soup = BS(html, "html.parser")
    #print(type(soup))
    for i in soup.find_all('div', class_="el"):
        if i.p:
            if i.p.attrs["class"][0] == "t1":
                
                position=i.p.span.text.strip()
                salary=i.find('span', class_="t4").text
                
                #find more inforamtion in the sub url
                suburl=i.p.span.a['href']
                #=
                try:
                    infor_list=get_sub_information(suburl)
                except Exception as e:
                    print(e)
                    infor_list=[]
                if len(infor_list)==6:
                    location,experience,education,numberofrecruitment,date,company_name=infor_list
                    info_list.append([position,company_name,location,experience,education,numberofrecruitment,date,salary])                    
                    print(num,infor_list)    
                else:
                    print("Information errors","--"*30)
                    print(num,position)
                num=num+1
    return num,info_list

def main(i): 
    num=i*10000
    all_infor_list=[]
    print("page: ",i)
    url="https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,{}.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=".format(i)
    try:
        html=get_html(url).decode("gbk")
        num,info_list=phase_html(html,num)
        all_infor_list+=info_list
        print("There are: ",len(all_infor_list))
        save_to_mysql(all_infor_list)
    except Exception as e:
        print(e)

#main()

In [35]:
def clear_table():
    conn=pymysql.connect(host="localhost",user="root",password="xs448058",db="h4",charset="utf8mb4")
    cursor=conn.cursor()
    insert_sql="TRUNCATE TABLE jobs511"
    #cursor.execute('SET NAMES utf8')
    try:
        cursor.execute(insert_sql)
    except Exception as e:
        print(e)
    conn.commit()
    cursor.close()
    conn.close()
clear_table()

In [31]:
from multiprocessing import Pool

In [37]:
#start=time.time()
pool=Pool(8)# 2 68 # 4 31 # 8 19 # 16 17 #32 20
pool.map(main,range(1,51))
pool.close()
pool.join()# 挂起
#end=time.time()
#print("{0}程序运行总时间为{1}秒{0}".format("-"*15,end-start))

page:  5
page:  3
page:  11
page:  9
page:  7
page:  1
page:  13
page:  15
130000 ['上海-浦东新区', '5-7年经验', '本科', '招5人', '11-26发布', '上海龙照电子有限公司']
110000 ['昆山', '1年经验', '大专', '招2人', '11-26发布', '昆山金裕支付联盟信息科技有限公司']
50000 ['长沙-芙蓉区', '5-7年经验', '本科', '招1人', '11-26发布', '北京开普云信息科技有限公司']
150000 ['杭州', '2年经验', '本科', '招1人', '11-26发布', '浙江大维高新技术股份有限公司']
10000 ['北京', '3-4年经验', '本科', '招1人', '11-26发布', '字节跳动']
90000 ['上海-闵行区', '无工作经验', '本科', '招2人', '11-26发布', '上海软科教育信息咨询有限公司']
30000 ['广州-黄埔区', '5-7年经验', '本科', '招1人', '11-26发布', '广州康大工业科技产业有限公司']
Information errors ------------------------------------------------------------
70000 朝九晚五 数据分析助理 双休
110001 ['东莞', '1年经验', '本科', '招若干人', '11-26发布', '东莞三星视界有限公司']
Information errors ------------------------------------------------------------
150001 大数据分析师
130001 ['广州-白云区', '无工作经验', '中专', '招3人', '11-26发布', '广州市爱莲化妆品有限公司']
90001 ['武汉-武昌区', '无工作经验', '中专', '招若干人', '11-26发布', '武汉人瑞人力资源服务有限公司']
30001 ['南通', '1年经验', '大专', '招1人', '11-26发布', '去哪儿网机票事业部']
50001 ['沈阳-皇姑区', '

30013 数据分析师
130014 ['重庆', '无工作经验', '大专', '招6人', '11-26发布', '西安铂石基金管理有限公司']
150014 ['昆明-西山区', '1年经验', '本科', '招若干人', '11-26发布', '昆明贝壳房地产经纪有限公司']
110014 ['广州', '无工作经验', '本科', '招若干人', '11-26发布', '广州通易科技有限公司']
70015 ['北京', '无工作经验', '硕士', '招1人', '11-26发布', '中国电信研究院']
10014 ['长沙-开福区', '3-4年经验', '本科', '招1人', '11-26发布', '湖南快乐通宝小额贷款有限公司']
50014 ['昆山', '3-4年经验', '本科', '招1人', '11-26发布', '昆山丘钛微电子科技有限公司']
90014 ['上海-浦东新区', '无工作经验', '本科', '招1人', '11-26发布', '上海邮乐网络技术有限公司']
30014 ['广州-番禺区', '3-4年经验', '大专', '招2人', '11-26发布', '广东安之伴实业有限公司']
Information errors ------------------------------------------------------------
130015 数据分析师
110015 ['青岛-李沧区', '无工作经验', '大专', '招4人', '11-26发布', '山东中达信通智能科技有限公司']
150015 ['广州-天河区', '2年经验', '招2人', '11-26发布', '城市规划 地理信息系统', '广东国地规划科技股份有限公司']
50015 ['上海-浦东新区', '2年经验', '本科', '招若干人', '11-26发布', '上海澳马信息技术服务有限公司']
70016 ['温州', '无工作经验', '本科', '招10人', '11-26发布', '福建省三福百货有限公司']
10015 ['北京', '无工作经验', '硕士', '招2人', '11-26发布', '北京清华同衡规划设计研究院有限公司']
90015 ['上海', '2年经验', '大专', '招1人', '

Information errors ------------------------------------------------------------
70030 临床数据分析员
90028 ['重庆-渝北区', '无工作经验', '大专', '招2人', '11-26发布', '成都风暴之灵蓝猫网络科技有限公司']
30028 ['潍坊', '无工作经验', '本科', '招若干人', '11-26发布', '潍柴动力股份有限公司']
110029 ['上海-黄浦区', '3-4年经验', '本科', '招1人', '11-26发布', '上海倍通数据集团']
10028 ['广州-天河区', '无工作经验', '大专', '招5人', '11-26发布', '广州固诚信息咨询服务有限公司']
50029 ['广州', '1年经验', '大专', '招3人', '11-26发布', '深圳优品科技有限公司']
150029 ['广州-天河区', '1年经验', '大专', '招1人', '11-26发布', '健合集团——广州总部']
30029 ['武汉', '无工作经验', '本科', '招若干人', '11-26发布', '中海物业管理有限公司']
70031 ['上海-长宁区', '1年经验', '本科', '招若干人', '11-27发布', '上海三迭信息科技有限公司']
130030 ['南京-鼓楼区', '1年经验', '大专', '招2人', '11-26发布', '南京方橙吾语商贸有限公司']
10029 ['深圳-南山区', '1年经验', '本科', '招1人', '11-26发布', 'OPPO']
Information errors ------------------------------------------------------------
90029 数据分析专员
110030 ['广州-天河区', '1年经验', '大专', '招1人', '11-26发布', '广州潭州教育网络科技有限公司']
50030 ['济南', '无工作经验', '本科', '招1人', '11-26发布', '山东舜德数据管理软件工程有限公司']
150030 ['武汉-洪山区', '无工作经验', '本科', '招若干人', '1

70045 ['深圳-龙岗区', '无工作经验', '本科', '招2人', '11-26发布', '深圳市时商创展科技有限公司']
130044 ['上海', '2年经验', '本科', '招2人', '11-26发布', '上海宏原信息科技有限公司']
110044 ['西安-雁塔区', '1年经验', '高中', '招若干人', '11-26发布', '西安澳信嘉诚商务咨询有限公司']
10043 ['广州-天河区', '2年经验', '大专', '招1人', '11-26发布', '广州千骐动漫有限公司']
150044 ['上海-静安区', '2年经验', '大专', '招1人', '11-26发布', '灰盒子（上海）咖啡有限公司']
30045 ['杭州', '无工作经验', '本科', '招2人', '11-26发布', '浙江银泰百货有限公司']
50044 ['西安-未央区', '1年经验', '大专', '招1人', '11-26发布', '陕西浩唐工贸有限公司']
90043 ['深圳-福田区', '3-4年经验', '大专', '招2人', '11-26发布', '深圳市赢时胜信息技术股份有限公司']
Information errors ------------------------------------------------------------
130045 数据分析主管/经理
70046 ['西安-高新技术产业开发区', '无工作经验', '本科', '招3人', '11-26发布', '西安明智康汇信息科技有限公司']
110045 ['上海', '2年经验', '本科', '招1人', '11-26发布', '上海道生医疗科技有限公司']
10044 ['广州', '2年经验', '大专', '招若干人', '11-26发布', '娇兰佳人集团总部']
150045 ['深圳', '3-4年经验', '大专', '招1人', '11-26发布', '深圳市联通达塑胶五金有限公司']
90044 ['长沙', '1年经验', '本科', '招1人', '11-26发布', '湖南奇网网络科技有限公司']
30046 ['杭州', '3-4年经验', '本科', '招若干人', '11-26发布', '阿里巴巴集团']
50

100007 ['广州', '3-4年经验', '本科', '招2人', '11-26发布', '广州华工邦元信息技术有限公司']
60008 ['成都', '无工作经验', '本科', '招5人', '11-26发布', '成都天府软件园有限公司']
120009 ['信阳', '3-4年经验', '本科', '招1人', '11-26发布', '广州三梅国际货运代理有限公司信阳分公司']
Information errors ------------------------------------------------------------
80010 数据分析师（金融技术岗）
40008 ['广州-天河区', '1年经验', '大专', '招2人', '11-26发布', '广东兆丰恒业控股集团有限公司']
Information errors ------------------------------------------------------------
140009 医用耗材数据分析专员
Information errors ------------------------------------------------------------
160009 数据分析师
100008 ['深圳-宝安区', '1年经验', '大专', '招若干人', '11-26发布', '深圳深国交投资发展有限公司']
60009 ['广州-白云区', '无工作经验', '本科', '招2人', '11-26发布', '广州广之旅国际旅行社股份有限公司']
Information errors ------------------------------------------------------------
80011 [多力葵花油]销售数据分析
Information errors ------------------------------------------------------------
120010 数据分析师
20008 ['上海-静安区', '1年经验', '大专', '招2人', '11-26发布', '广东壹号食品股份有限公司']
40009 ['上海-长宁区', '2年经验', '本科', '招若干人', '11-26发布',

...

Information errors ------------------------------------------------------------
430042 网站运营经理
390044 ['上海', '3-4年经验', '大专', '招1人', '11-25发布', '乐扣乐扣（中国）']
330047 ['深圳-南山区', '无工作经验', '本科', '招3人', '11-26发布', '中南教育科技（深圳）有限公司']
350046 ['西安-雁塔区', '3-4年经验', '本科', '招1人', '11-26发布', '西安万兴服务外包有限责任公司']
410045 ['杭州-江干区', '1年经验', '本科', '招若干人', '10-09发布', '温州市鹿城区中津先进科技研究院']
470042 ['郑州-金水区', '无工作经验', '大专', '招5人', '11-26发布', '郑州嘉尚电子科技有限公司']
370046 ['深圳-宝安区', '1年经验', '大专', '招2人', '11-26发布', '鹏鼎控股（深圳）股份有限公司']
450043 ['广州-海珠区', '1年经验', '大专', '招2人', '11-26发布', '广州纽得赛生物科技有限公司']
430043 ['北京-朝阳区', '2年经验', '本科', '招3人', '11-26发布', '英雄互娱科技股份有限公司']
Information errors ------------------------------------------------------------
410046 数据分析师G00847
390045 ['上海-徐汇区', '2年经验', '大专', '招1人', '11-25发布', '阿甘锅盔']
350047 ['广州', '1年经验', '本科', '招3人', '11-26发布', '广州天维信息技术股份有限公司']
330048 ['广州-天河区', '2年经验', '大专', '招1人', '11-26发布', '美尚(广州)化妆品股份有限公司']
470043 ['深圳-龙岗区', '3-4年经验', '招1人', '11-26发布', '英语熟练', '深圳市乐骑智能科技有限公司']
370047 [

380009 数据分析专员
460006 ['上海', '5-7年经验', '本科', '招1人', '11-26发布', '上海艾力斯医药科技有限公司']
440006 ['泸州', '1年经验', '大专', '招10人', '11-26发布', '四川成都贝壳闹海房地产经纪有限公司']
360010 ['上海', '1年经验', '大专', '招4人', '11-26发布', '上海邦成达资产管理有限公司']
480006 ['深圳-宝安区', '1年经验', '大专', '招3人', '11-26发布', '前海行者悟科技（深圳）有限公司']
400008 ['西安', '1年经验', '大专', '招2人', '11-25发布', '中节能（西安）监测技术有限公司']
420010 ['上海-长宁区', '5-7年经验', '本科', '招1人', '11-26发布', '携程旅游信息技术（上海）有限公司']
380010 ['嘉兴', '3-4年经验', '本科', '招1人', '11-26发布', '浙江亚特电器有限公司']
340011 ['北京-丰台区', '3-4年经验', '本科', '招3人', '11-26发布', '北京万泰中联科技股份有限公司']
460007 ['广州-天河区', '2年经验', '大专', '招2人', '11-26发布', '耐力运动科技（广州）有限公司']
360011 ['广州-番禺区', '2年经验', '本科', '招1人', '11-26发布', '广东长隆集团有限公司']
440007 ['上海-浦东新区', '无工作经验', '本科', '招20人', '11-26发布', '上海丽兹行-SH03']
Information errors ------------------------------------------------------------
480007 业务架构师(BA)
420011 ['重庆-巫山县', '5-7年经验', '大专', '招1人', '11-26发布', '农夫山泉股份有限公司']
400009 ['佛山', '无工作经验', '硕士', '招5人', '11-25发布', '佛山市城市安全研究中心有限公司']
340012 ['大连-甘井子区', '无工作经

340024 ['郑州-中牟县', '无工作经验', '大专', '招1人', '11-26发布', '河南新鲜时刻果业有限公司']
440019 ['上海-长宁区', '5-7年经验', '本科', '招1人', '11-26发布', '苏州乐贝网络科技有限公司上海分公司']
380026 ['合肥-经开区', '2年经验', '本科', '招1人', '11-26发布', '联合利华（中国）有限公司']
360024 ['杭州-拱墅区', '3-4年经验', '本科', '招2人', '11-26发布', '浙江汇信科技有限公司']
460020 ['深圳-龙岗区', '无工作经验', '本科', '招1人', '11-26发布', '广东百事泰电子商务股份有限公司']
400021 ['北京-朝阳区', '2年经验', '本科', '招1人', '11-25发布', '爱尔康（中国）眼科产品有限公司']
'gbk' codec can't decode byte 0xaf in position 111: illegal multibyte sequence
Information errors ------------------------------------------------------------
380027 Product Sales Data Analyst 数据分析专员  3335930
420024 ['成都', '2年经验', '大专', '招1人', '11-26发布', 'LG生活健康（化妆品、日用品）成都分公司']
340025 ['南京-浦口区', '2年经验', '本科', '招3人', '11-26发布', '南京雪糕网络科技有限公司']
480020 ['重庆-九龙坡区', '无工作经验', '中技', '招9人', '11-26发布', '名鹏信息技术有限公司']
440020 ['深圳-南山区', '3-4年经验', '本科', '招2人', '11-26发布', '深圳前海浩方科技有限公司']
360025 ['长沙-芙蓉区', '2年经验', '本科', '招1人', '11-26发布', '澳优乳业（中国）有限公司——纽莱可事业部']
400022 ['上海-静安区', '无工作经验', '本科', '招若

400035 ['武汉-江夏区', '5-7年经验', '大专', '招1人', '11-25发布', '武汉小药药医药科技有限公司']
340038 ['北京-海淀区', '3-4年经验', '本科', '招若干人', '11-26发布', '北京连盈科技有限公司']
Information errors ------------------------------------------------------------
480033 媒介采购专员
360037 ['鄂州', '1年经验', '本科', '招1人', '11-26发布', '湖北九城文化传播有限公司']
440034 ['上海-徐汇区', '3-4年经验', '大专', '招1人', '11-26发布', '上海深艾信息科技有限公司']
420038 ['呼和浩特', '1年经验', '本科', '招1人', '11-26发布', '无锡小天鹅股份有限公司']
460034 ['广州-白云区', '无工作经验', '本科', '招5人', '11-26发布', '广州市沃芙网络科技有限公司']
Information errors ------------------------------------------------------------
380041 LCM Specialist(数据分析）
400036 ['中山', '2年经验', '本科', '招1人', '11-25发布', '中燃宝电气（深圳）有限公司']
340039 ['上海-徐汇区', '3-4年经验', '本科', '招1人', '11-26发布', '上海美市科技有限公司']
360038 ['上海-闵行区', '2年经验', '本科', '招1人', '11-26发布', '上海颗纯信息技术有限公司']
480034 ['深圳', '1年经验', '大专', '招5人', '11-26发布', '深圳市威斯达照明有限公司']
440035 ['南京', '无工作经验', '大专', '招若干人', '11-26发布', '南京链家房地产经纪有限公司']
420039 ['深圳-坪山区', '3-4年经验', '大专', '招1人', '11-26发布', '深圳齐心集团股份有限公司']
380042 ['武汉

440049 ['深圳-龙华新区', '2年经验', '大专', '招1人', '11-26发布', '深圳市卡斐乐科技有限公司']
There are:  46
480048 ['广州-天河区', '无工作经验', '本科', '招2人', '11-26发布', '金地（集团）广州地产公司']
490004 ['武汉-洪山区', '无工作经验', '硕士', '招1人', '11-26发布', '亚科因（武汉）生物技术有限公司']
460049 ['东莞', '5-7年经验', '大专', '招1人', '11-26发布', '东莞南城保时捷中心']
There are:  40
480049 ['南昌', '2年经验', '大专', '招2人', '11-26发布', '江西省蓝源科技股份有限公司']
There are:  34
490005 ['杭州-江干区', '1年经验', '大专', '招1人', '11-26发布', '浙江途骜网络科技有限责任公司']
490006 ['成都-金牛区', '2年经验', '大专', '招8人', '11-26发布', '平安普惠信息服务有限公司成都人民北路分公司']
Information errors ------------------------------------------------------------
490007 资深文案策划
490008 ['广州', '无工作经验', '大专', '招1人', '11-26发布', '北京万华恒信信息技术有限公司']
490009 ['郑州-高新区', '无工作经验', '大专', '招2人', '11-26发布', '河南郑矿机器有限公司']
490010 ['广州-天河区', '1年经验', '大专', '招4人', '11-26发布', '广州时科信息技术有限公司']
490011 ['广州-天河区', '1年经验', '本科', '招2人', '11-26发布', '广州酷麦信息科技有限公司']
490012 ['广州-番禺区', '1年经验', '大专', '招2人', '11-26发布', '领积信息技术（广州）有限公司']
490013 ['北京-朝阳区', '1年经验', '大专', '招1人', '11-26发布', '本来生活']
49